# Bank Loan Algorithm

Importing all the Necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
#do i need to have a random seed here

from sklearn.preprocessing import StandardScaler


%matplotlib inline

Importing the data into a dataframe and viewing its content

In [2]:
train = pd.read_csv("data/madfhantr.csv")
test= pd.read_csv("data/madhante.csv") # combined?

In [3]:
train.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [5]:
train.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [6]:
train.info


<bound method DataFrame.info of       Loan_ID  Gender Married Dependents     Education Self_Employed  \
0    LP001002    Male      No          0      Graduate            No   
1    LP001003    Male     Yes          1      Graduate            No   
2    LP001005    Male     Yes          0      Graduate           Yes   
3    LP001006    Male     Yes          0  Not Graduate            No   
4    LP001008    Male      No          0      Graduate            No   
..        ...     ...     ...        ...           ...           ...   
609  LP002978  Female      No          0      Graduate            No   
610  LP002979    Male     Yes         3+      Graduate            No   
611  LP002983    Male     Yes          1      Graduate            No   
612  LP002984    Male     Yes          2      Graduate            No   
613  LP002990  Female      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0               5849          

This shows us the number of approved vs those who havent been approved

In [7]:
train['Loan_Status'].value_counts() # that will play a factor to determine which metric we use 

Y    422
N    192
Name: Loan_Status, dtype: int64

Lets identify the missing values

In [8]:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [9]:
test.isnull().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

We need to deal with the null values

is there a training and test for this data set futher investigation needed

We are going to replace the missing values with the most common numbers and categories (The  mode)

We are going to create a fucntion to turn replace the null values with the mode the most common number. 

In [10]:
def remove_null(column,df): # rename fill null 
    df[column].fillna(df[column].mode()[0], inplace= True) # mode instead of mean

Looping thotugh the categorical columns using the function we built 

In [11]:
categorical_columns = ['Gender','Married','Dependents','Self_Employed','Credit_History']

for column in categorical_columns:
    remove_null(column, train)

In [12]:
for column in categorical_columns:
    remove_null(column, test)

In [13]:
test.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           5
Loan_Amount_Term     6
Credit_History       0
Property_Area        0
dtype: int64

now lets transform numerical varialbles for this one instead of taking the most common one we will take the average? 

In [14]:
numerical_columns = [ 'LoanAmount', 'Loan_Amount_Term']
for column in numerical_columns:
    remove_null(column, train) # we still took the mode we should check if creating a function to take the average will improve accuracy

In [15]:
for column in numerical_columns:
    remove_null(column, test)

In [16]:
test.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

Now we are now going to transform the categoricals and encode them as the algorightm understands numbers 

In [17]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,120.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [18]:
le= preprocessing.LabelEncoder()
train['Loan_Status'] = le.fit_transform(train['Loan_Status']) 

In [19]:
#def encode_features(df): # said I needed a random state
features = ['Gender','Married','Education','Self_Employed','Property_Area'] #add loan status here 
    
for feature in features: # whats the difference between OHE and Label encoder 
    le.fit(train[feature])
    train[feature]= le.transform(train[feature])
    test[feature]= le.transform(test[feature])
#    return df
#train = encode_features(train)
#test = encode_features(test) # 

#le = preprocessing.LabelEncoder()
# does it refit and transform it over here

train.head()    

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,0,0,5849,0.0,120.0,360.0,1.0,2,1
1,LP001003,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,LP001005,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,LP001006,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,LP001008,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1


In [20]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,1,1,0,0,0,5720,0,110.0,360.0,1.0,2
1,LP001022,1,1,1,0,0,3076,1500,126.0,360.0,1.0,2
2,LP001031,1,1,2,0,0,5000,1800,208.0,360.0,1.0,2
3,LP001035,1,1,2,0,0,2340,2546,100.0,360.0,1.0,2
4,LP001051,1,0,0,1,0,3276,0,78.0,360.0,1.0,2


Lets check all the data types 

We are going to try and combine applicant income to coaplicant income and also the monthly installment

In [21]:
#train["Total_Income"]= train['ApplicantIncome'] + train['CoapplicantIncome']
#train['Monthly_Installments']= train['LoanAmount']/train['Loan_Amount_Term']

#test["Total_Income"]= test['ApplicantIncome'] + test['CoapplicantIncome']
#test['Monthly_Installments']= test['LoanAmount']/test['Loan_Amount_Term']

#Did this really help

In [22]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,0,0,5849,0.0,120.0,360.0,1.0,2,1
1,LP001003,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,LP001005,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,LP001006,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,LP001008,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1


In [23]:
train.dtypes

Loan_ID               object
Gender                 int32
Married                int32
Dependents            object
Education              int32
Self_Employed          int32
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          int32
Loan_Status            int32
dtype: object

We can drop Loan ID but I will do that later while imputing the data frame but we need to change the dependents froom a strings to numbers 

In [24]:
data_frames= [train, test]
for df in data_frames:
    df['Dependents'].replace({'0': 0, '1': 1, '2': 2, '3+':3}, inplace = True)

In [25]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,1,1,0,0,0,5720,0,110.0,360.0,1.0,2
1,LP001022,1,1,1,0,0,3076,1500,126.0,360.0,1.0,2
2,LP001031,1,1,2,0,0,5000,1800,208.0,360.0,1.0,2
3,LP001035,1,1,2,0,0,2340,2546,100.0,360.0,1.0,2
4,LP001051,1,0,0,1,0,3276,0,78.0,360.0,1.0,2


Now we can start modeling

In [26]:
X= train.drop(['Loan_ID', 'Loan_Status'], axis= 1 )
y= train['Loan_Status'] #where can i use the test data set

Scale the model 

We are scaling the data to make sure everything is on the same scale

Splitting it into train test split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .25, random_state=42)

In [29]:
scaler= StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [30]:
len(y)

614

Now we can start using te different classifying algorithms lets start with the decision tree model.


In [31]:
dtcf= DecisionTreeClassifier(random_state=42) # do we have to put a random state here?
dtcf.fit(X_train_scaled, y_train)

DecisionTreeClassifier(random_state=42)

In [32]:
dtcf_pred= dtcf.predict(X_test_scaled)

In [33]:
accuracy_score(y_test, dtcf_pred) # so here we get our very first performance on the model

0.6753246753246753

Lets see if we can improve the model with grid search

In [35]:
grid= ({'max_depth': [4, 5, 6],
        'min_samples_split': [6,12,16]}) #how did we pick out he numbers for the Decision Tree


grid_dt= GridSearchCV(dtcf, param_grid= grid, scoring= 'accuracy', cv=3, verbose=1) # find out the role of cv and verbose

grid_dt.fit(X_train_scaled,y_train)

print(grid_dt.best_score_)

best_dt_estimator= grid_dt.best_estimator_
print(best_dt_estimator)
best_dt_estimator_pred= best_dt_estimator.predict(X_test_scaled)
accuracy_score(y_test, best_dt_estimator_pred) #why is the score and the accuracy different

Fitting 3 folds for each of 9 candidates, totalling 27 fits
0.8130181365475483
DecisionTreeClassifier(max_depth=4, min_samples_split=6, random_state=42)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    0.0s finished


0.7272727272727273

Now lets try the random forest classifier

use dummy classifier to use as baseline model 

In [36]:
rdf= RandomForestClassifier()

In [37]:
rdf.fit(X_train_scaled, y_train,) # revisit the sample weight

RandomForestClassifier()

In [38]:
rdf_pred= rdf.predict(X_test_scaled)
accuracy_score(y_test, rdf_pred) 

0.7727272727272727

we got a better original score as expected of the random forest model lets see if we can tune some of its parameters to be even more accurate


In [39]:
grid_rdf_params= ({'n_estimators': [100,150,200],
           'max_depth': [4,5,6],
           'min_samples_leaf': [6,10,14], # why these numbers
            'min_samples_split': [4,5,6,7]})

grid_rdf= GridSearchCV(rdf, param_grid= grid_rdf_params, scoring='accuracy', cv= 3, verbose=1)

grid_rdf.fit(X_train, y_train)




Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 324 out of 324 | elapsed:   48.3s finished


GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [4, 5, 6],
                         'min_samples_leaf': [6, 10, 14],
                         'min_samples_split': [4, 5, 6, 7],
                         'n_estimators': [100, 150, 200]},
             scoring='accuracy', verbose=1)

In [40]:
print(grid_rdf.best_score_)

0.8217044393514982


In [41]:
best_rdf_estimator= grid_rdf.best_estimator_

best_rdf_estimator_pred= best_rdf_estimator.predict(X_test)
accuracy_score(y_test, best_rdf_estimator_pred) #why is that not 82

0.7727272727272727

Doesnt seem like it helped why is that

Now lets try the XGBoost Classifier. 

In [ ]:
have a confusion matrix decide which metric is most important ( classification report)

In [42]:
import xgboost as xgb
from xgboost import XGBClassifier

In [43]:
xgb= XGBClassifier(n_estimator=500, learning_rate=0.05, max_depth=3) # why did we choose these parameters 
evals= [(X_test, y_test)]
xgb.fit(X_train, y_train,
    eval_set=evals,
    eval_metric="error", #what does error do 
    verbose= True)


[13:36:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-error:0.24026
[1]	validation_0-error:0.24026
[2]	validation_0-error:0.24026
[3]	validation_0-error:0.24026
[4]	validation_0-error:0.24026
[5]	validation_0-error:0.24026
[6]	validation_0-error:0.24026
[7]	validation_0-error:0.24026
[8]	validation_0-error:0.24026
[9]	validation_0-error:0.22727
[10]	validation_0-error:0.22727
[11]	validation_0-error:0.22727
[12]	validation_0-error:0.22727
[13]	validation_0-error:0.22727
[14]	validation_0-error:0.22727
[15]	validation_0-error:0.22727
[16]	validation_0-error:0.22727
[17]	validation_0-error:0.22727
[18]	validation_0-error:0.22727
[19]	validati

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimator=500, n_estimators=100, n_jobs=0, num_parallel_tree=1,
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [46]:
xgb_pred= xgb.predict(X_test)
accuracy_score(y_test, xgb_pred)

0.7857142857142857

XGBoost improved it a little bit

look at the feature importance adn have the top 5 features. 